# Tools for NLP

In [2]:
import findspark

In [3]:
findspark.init('/home/oussama/spark-2.4.0-bin-hadoop2.7')

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [7]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [8]:
sentenceData = spark.createDataFrame([
    (0.0, 'hi I heared about spark'),
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic,regression,models,are,neat')
],['label', 'sentence'])

In [9]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|hi I heared about...|
|  0.0|I wish java could...|
|  1.0|Logistic,regressi...|
+-----+--------------------+



In [10]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [11]:
words_data = tokenizer.transform(sentenceData)

In [12]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|hi I heared about...|[hi, i, heared, a...|
|  0.0|I wish java could...|[i, wish, java, c...|
|  1.0|Logistic,regressi...|[logistic,regress...|
+-----+--------------------+--------------------+



In [14]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [15]:
featurized_data = hashing_tf.transform(words_data)

In [16]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [17]:
idf_model = idf.fit(featurized_data)

In [19]:
rescaled_data = idf_model.transform(featurized_data)

In [21]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[24417,49304,91137,234657,254577],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                    |
|0.0  |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [22]:
from pyspark.ml.feature import CountVectorizer

In [23]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [24]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [26]:
cv = CountVectorizer(inputCol='words', outputCol='features',
                    vocabSize=3,minDF=2.0)

In [29]:
model = cv.fit(df)

In [30]:
result = model.transform(df)

In [31]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

